# ==============================================================================
# NOTEBOOK COMPLET : AGENT DE TRADING DQN ASSISTÉ PAR UN MODÈLE TFT
# ==============================================================================

# ------------------------------------------------------------------------------
# SECTION 0 : INSTALLATION ET IMPORTATION DES BIBLIOTHÈQUES
# ------------------------------------------------------------------------------
# On installe les bibliothèques nécessaires, y compris celles pour le TFT


In [8]:
!pip install pandas
!pip install numpy 
!pip install scikit-learn 
!pip install tensorflow 
!pip install gymnasium 
!pip install stable-baselines3 
!pip install pytorch-forecasting 
!pip install pandas-ta 
!pip install pytorch-lightning -q


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import

In [9]:
import pandas as pd
import numpy as np
import glob
import random
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

# Bibliothèques pour le TFT
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss
import pandas_ta as ta

# Bibliothèques pour l'agent RL
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import DQN



# ==============================================================================
# PARTIE 1 : CHARGEMENT ET INGÉNIERIE DES CARACTÉRISTIQUES (FEATURES)
# ==============================================================================

In [13]:
print("\n--- PARTIE 1 : Chargement et enrichissement des données ---")

# --- 1.1 Chargement des fichiers ---
folder_path = 'Global Stock Market (2008-2023)/*.csv'
all_files = glob.glob(folder_path)
df_list = [pd.read_csv(filename) for filename in all_files]
df_full = pd.concat(df_list, ignore_index=True)

# --- 1.2 Ajout des indicateurs et nettoyage des noms de colonnes ---
data_by_ticker_featured = {}
tickers_to_use = df_full['Ticker'].unique()

for ticker in tickers_to_use:
    df_ticker = df_full[df_full['Ticker'] == ticker].copy()
    if len(df_ticker) < 252: # S'assurer d'avoir au moins 1 an de données
        continue
    
    df_ticker['Date'] = pd.to_datetime(df_ticker['Date'])
    df_ticker.sort_values('Date', inplace=True)
    df_ticker.set_index('Date', inplace=True)
    
    # Ajout des indicateurs techniques
    df_ticker.ta.sma(length=20, append=True)
    df_ticker.ta.rsi(length=14, append=True)
    df_ticker.ta.bbands(length=20, append=True)
    
    # Caractéristiques temporelles
    df_ticker['day_of_week'] = df_ticker.index.dayofweek
    df_ticker['month'] = df_ticker.index.month
    df_ticker['year'] = df_ticker.index.year
    
    df_ticker.fillna(method='bfill', inplace=True)
    
    # === CORRECTION CRUCIALE ===
    # Nettoyer les noms de colonnes pour enlever les points '.'
    df_ticker.columns = [col.replace('.', '_') for col in df_ticker.columns]
    
    data_by_ticker_featured[ticker] = df_ticker

print(f"Données enrichies et nettoyées chargées pour {len(data_by_ticker_featured)} tickers.")
print("Exemple de colonnes après nettoyage:", next(iter(data_by_ticker_featured.values())).columns.tolist())


--- PARTIE 1 : Chargement et enrichissement des données ---
Données enrichies et nettoyées chargées pour 12 tickers.
Exemple de colonnes après nettoyage: ['Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'SMA_20', 'RSI_14', 'BBL_20_2_0_2_0', 'BBM_20_2_0_2_0', 'BBU_20_2_0_2_0', 'BBB_20_2_0_2_0', 'BBP_20_2_0_2_0', 'day_of_week', 'month', 'year']


C:\Users\anase\AppData\Local\Temp\ipykernel_19540\3087193200.py:32: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_ticker.fillna(method='bfill', inplace=True)
C:\Users\anase\AppData\Local\Temp\ipykernel_19540\3087193200.py:32: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_ticker.fillna(method='bfill', inplace=True)
C:\Users\anase\AppData\Local\Temp\ipykernel_19540\3087193200.py:32: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_ticker.fillna(method='bfill', inplace=True)
C:\Users\anase\AppData\Local\Temp\ipykernel_19540\3087193200.py:32: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_ticker.fillna(method='bfill', in

# ==============================================================================
# PARTIE 2 : PRÉDICTION AVEC LE TEMPORAL FUSION TRANSFORMER (TFT)
# ==============================================================================

In [14]:
print("\n--- PARTIE 2 : Prédiction avec le Temporal Fusion Transformer (TFT) ---")

# --- 2.1 Formatage global des données pour Pytorch Forecasting ---
print("Formatage des données pour le TFT...")
all_featured_dfs = []
for ticker, df_feat in data_by_ticker_featured.items():
    df_temp = df_feat.reset_index().copy()
    df_temp['group'] = ticker
    all_featured_dfs.append(df_temp)
df_for_tft = pd.concat(all_featured_dfs, ignore_index=True)
df_for_tft['time_idx'] = df_for_tft.groupby('group').cumcount()

# --- 2.2 Création du TimeSeriesDataSet ---
max_encoder_length = 60
max_prediction_length = 1
training_cutoff = df_for_tft['time_idx'].max() - 252

training_data = TimeSeriesDataSet(
    df_for_tft[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="Close",
    group_ids=["group"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["group"],
    time_varying_known_reals=["time_idx", "day_of_week", "month", "year"],
    # === CORRECTION CRUCIALE ===
    # Utiliser les noms de colonnes nettoyés (sans le '.0')
    time_varying_unknown_reals=[
        'Open', 'High', 'Low', 'Volume', 'SMA_20', 'RSI_14',
        'BBL_20_2_0', 'BBM_20_2_0', 'BBU_20_2_0', 'BBB_20_2_0', 'BBP_20_2_0'
    ],
    target_normalizer=GroupNormalizer(groups=["group"], transformation="softplus"),
    add_relative_time_idx=True, add_target_scales=True, add_encoder_length=True,
)

validation_data = TimeSeriesDataSet.from_dataset(training_data, df_for_tft, predict=True, stop_randomization=True)
train_dataloader = training_data.to_dataloader(train=True, batch_size=128, num_workers=0)
val_dataloader = validation_data.to_dataloader(train=False, batch_size=256, num_workers=0)

# --- 2.3 Entraînement du modèle TFT ---
# (Cette partie reste la même)
print("Entraînement du modèle TFT...")
pl.seed_everything(42)
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=5, verbose=False, mode="min")
trainer = pl.Trainer(max_epochs=20, accelerator="auto", gradient_clip_val=0.1, limit_train_batches=30, callbacks=[early_stop_callback])
tft = TemporalFusionTransformer.from_dataset(training_data, learning_rate=0.03, hidden_size=32, attention_head_size=2, dropout=0.1, hidden_continuous_size=16, loss=QuantileLoss(), optimizer="Ranger")
trainer.fit(tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

# --- 2.4 Génération des prédictions ---
# (Cette partie reste la même)
print("Génération des prédictions du TFT pour tout le dataset...")
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
predictions = raw_predictions["prediction"][:, :, 3].numpy().flatten()

predictions_df = pd.DataFrame({'tft_prediction': predictions, 'group': x['group_ids'].flatten(), 'time_idx': x['decoder_time_idx'].flatten()})
df_final_with_preds = pd.merge(df_for_tft, predictions_df, on=['group', 'time_idx'], how='left')
df_final_with_preds['tft_prediction'] = df_final_with_preds.groupby('group')['tft_prediction'].transform(lambda x: x.bfill().ffill())
df_final_with_preds.dropna(inplace=True)

# --- Création du dictionnaire final pour l'agent RL ---
# Cette variable sera utilisée plus tard, ce qui résout la NameError
data_by_ticker_final = {}
for ticker in df_final_with_preds['group'].unique():
    df_ticker = df_final_with_preds[df_final_with_preds['group'] == ticker].copy()
    df_ticker.set_index('Date', inplace=True)
    data_by_ticker_final[ticker] = df_ticker

print("Aperçu du DataFrame final enrichi pour un ticker :")
print(next(iter(data_by_ticker_final.values())).head())


--- PARTIE 2 : Prédiction avec le Temporal Fusion Transformer (TFT) ---
Formatage des données pour le TFT...


KeyError: "None of [Index(['BBL_20_2_0'], dtype='object')] are in the [columns]"

# ==============================================================================
# PARTIE 3 : APPRENTISSAGE PAR RENFORCEMENT AVEC DQN
# ==============================================================================

In [ ]:
print("\n--- PARTIE 3 : Définition de l'environnement et entraînement du DQN ---")

# --- 3.1 Environnement de Trading ---
class MultiStockTradingEnv(gym.Env):
    def __init__(self, data_dict, initial_balance=10_000):
        super().__init__()
        self.data_dict = data_dict
        self.tickers = list(data_dict.keys())
        self.initial_balance = initial_balance
        self.action_space = spaces.Discrete(4)

        sample_data = next(iter(self.data_dict.values()))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(len(sample_data.columns) + 2,), dtype=np.float32)

    def _select_new_stock(self):
        self.current_ticker = random.choice(self.tickers)
        self.data = self.data_dict[self.current_ticker].reset_index(drop=True)

    def reset(self, seed=None):
        super().reset(seed=seed)
        self._select_new_stock()
        self.balance = self.initial_balance
        self.shares_held = 0
        self.borrowed_balance = 0
        self.net_worth = self.initial_balance
        self.current_step = 0
        return self._get_observation(), {}

    def _get_observation(self):
        obs_features = self.data.iloc[self.current_step].values
        obs = np.concatenate([obs_features, [self.balance], [self.shares_held]]).astype(np.float32)
        obs[np.isinf(obs) | np.isnan(obs)] = 0
        return obs

    def step(self, action):
        previous_net_worth = self.net_worth
        current_price = self.data['Close'].iloc[self.current_step]
        leverage_factor = 2.0

        if action == 1 and self.balance > current_price: # Buy
            self.shares_held += 1; self.balance -= current_price
        elif action == 2 and self.shares_held > 0: # Sell
            self.shares_held -= 1; money_from_sale = current_price
            if self.borrowed_balance > 0:
                repayment = min(money_from_sale, self.borrowed_balance)
                self.borrowed_balance -= repayment
                self.balance += (money_from_sale - repayment)
            else: self.balance += money_from_sale
        elif action == 3 and self.balance > 0: # Leverage
            shares_to_buy = int((self.balance * leverage_factor) // current_price)
            if shares_to_buy > 0:
                cost = shares_to_buy * current_price
                self.shares_held += shares_to_buy
                self.borrowed_balance += cost - self.balance
                self.balance = 0

        self.net_worth = (self.balance + self.shares_held * current_price) - self.borrowed_balance
        reward = self.net_worth - previous_net_worth
        
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        if self.net_worth <= self.initial_balance * 0.5: done = True
            
        return self._get_observation(), reward, done, False, {}

    def render(self, mode='human'):
        print(f'Ticker: {self.current_ticker}, Step: {self.current_step}, Net Worth: {self.net_worth:.2f}')

# --- 3.2 Entraînement de l'agent DQN ---
env_train = MultiStockTradingEnv(data_dict=data_by_ticker_final)
model_dqn = DQN("MlpPolicy", env_train, verbose=1, tensorboard_log="./dqn_tft_tensorboard/")
model_dqn.learn(total_timesteps=200_000, progress_bar=True)
model_dqn.save("dqn_tft_trader")


# ==============================================================================
# PARTIE 4 : ÉVALUATION ET BACKTESTING
# ==============================================================================

In [ ]:
# ==============================================================================
# PARTIE 4 (CORRIGÉE) : Entraînement de l'agent DQN
# ==============================================================================

print("\n--- PARTIE 4 : Entraînement de l'agent DQN sur les données enrichies par le TFT ---")

# --- CORRECTION DE L'ERREUR 'NameError' ---
# On instancie l'environnement en utilisant 'data_by_ticker_final', qui a été
# créé à la fin de la cellule précédente.
env_train = MultiStockTradingEnv(data_dict=data_by_ticker_final)

model_dqn = DQN(
    "MlpPolicy",
    env_train,
    verbose=1,
    tensorboard_log="./dqn_tft_tensorboard/",
    exploration_fraction=0.5,
    exploration_final_eps=0.05,
    learning_rate=0.0005,
    buffer_size=100_000
)
model_dqn.learn(total_timesteps=200_000, progress_bar=True)
model_dqn.save("dqn_tft_trader")

In [ ]:



# --- Lancer TensorBoard ---
%load_ext tensorboard
%tensorboard --logdir ./dqn_tft_tensorboard/